In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers safetensors torch

In [ ]:
MODEL_DIR = "/content/drive/Shareddrives/cs685/mlm_bert_goemotions_biomed_finance"

In [ ]:
!pip install -q --upgrade "transformers>=4.30.0" datasets safetensors pyyaml accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 29.4 MB/s eta 0:00:00


In [ ]:
import transformers
print(transformers.__version__)

4.57.3


In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(MODEL_DIR)
print("model_type:", config.model_type)
print("architectures:", config.architectures)

model_type: bert
architectures: ['BertForMaskedLM']


In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,  # change if config says something else
)
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
model.to(device)
model.eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/Shareddrives/cs685/mlm_bert_goemotions_biomed_finance and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
!pip install -q transformers safetensors torch


In [ ]:
id2label = {
    0: "negative",
    1: "neutral",
    2: "positive",
}

In [ ]:
!pip install -q transformers datasets safetensors pyyaml accelerate


In [ ]:
from collections import Counter
from pathlib import Path

import yaml
import torch
from datasets import load_dataset, concatenate_datasets
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)



In [ ]:
def load_config(path: str):
    with open(path, "r") as f:
        return yaml.safe_load(f)


def load_jsonl_datasets(files):
    """Load one or more JSONL files into a single HF Dataset."""
    if isinstance(files, str):
        files = [files]
    dsets = [load_dataset("json", data_files=f, split="train") for f in files]
    return dsets[0] if len(dsets) == 1 else concatenate_datasets(dsets)


from transformers import Trainer
import torch

class WeightedTrainer(Trainer):
    """Trainer that applies a class-weighted cross-entropy loss."""

    def __init__(self, class_weights=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # tensor shape: [num_labels]
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        Accept **kwargs so we don't blow up if Trainer passes extra args
        like num_items_in_batch (newer versions). Older versions will just
        ignore **kwargs, which is also fine.
        """
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        if self.class_weights is not None:
            weight = self.class_weights.to(logits.device)
            loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        else:
            loss_fct = torch.nn.CrossEntropyLoss()

        loss = loss_fct(
            logits.view(-1, self.model.config.num_labels),
            labels.view(-1),
        )
        return (loss, outputs) if return_outputs else loss



In [ ]:
!pip install -q peft


In [ ]:
config = {
    "model_name_or_path": "/content/drive/Shareddrives/cs685/mlm_bert_goemotions_biomed_finance",
    "output_dir": "/content/drive/MyDrive/models/sft7_lora",
    "num_labels": 3,

    "train_files": [
        "/content/drive/Shareddrives/cs685/final_data_SFT/label_mixed_3_train.jsonl",
    ],
    "eval_files": [
        "/content/drive/Shareddrives/cs685/final_data_SFT/label_mixed_3_val.jsonl",
    ],

    # LoRA-friendly hyperparams
    "learning_rate": 1e-4,
    "batch_size": 16,
    "epochs": 10,

    "use_lora": True,
    "use_class_weights": True,

    # LoRA config
    "lora_r": 64,
    "lora_alpha": 128,
    "lora_dropout": 0.1,
}

In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# -------- read config values --------
model_path = config["model_name_or_path"]
output_dir = config["output_dir"]
num_labels = int(config["num_labels"])
use_lora = bool(config.get("use_lora", False))
use_class_weights = bool(config.get("use_class_weights", True))

tokenizer = AutoTokenizer.from_pretrained(model_path)

# 1) Add domain tags as special tokens
special_tokens_dict = {"additional_special_tokens": ["[FIN]", "[BIO]"]}
num_added = tokenizer.add_special_tokens(special_tokens_dict)
print("Added tokens:", num_added, tokenizer.additional_special_tokens)

# ----- base model -----
base_model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=num_labels,
    ignore_mismatched_sizes=True,
)

# 2) Resize embeddings so the model can use the new tokens
if num_added > 0:
    base_model.resize_token_embeddings(len(tokenizer))

for name, param in base_model.named_parameters():
    param.requires_grad = False

# ----- wrap with LoRA (or not) -----
if use_lora:
    lora_cfg = LoraConfig(
        task_type=TaskType.SEQ_CLS,                          # sequence classification
        r=int(config.get("lora_r", 32)),
        lora_alpha=int(config.get("lora_alpha", 64)),
        lora_dropout=float(config.get("lora_dropout", 0.05)),
        bias="none",
        target_modules=["query", "key", "value"],            # BERT attention projections
    )
    model = get_peft_model(base_model, lora_cfg)
    model.print_trainable_parameters()
else:
    model = base_model

# -------- load data --------
train_files = config["train_files"]
eval_files = config["eval_files"]

train_ds = load_jsonl_datasets(train_files)
eval_ds = load_jsonl_datasets(eval_files)

def encode(ex):
    domain = ex.get("domain", None)
    if domain == "FIN":
        prefix = "[FIN]"
    elif domain == "BIO":
        prefix = "[BIO]"
    else:
        prefix = ""

    text = f"{prefix} {ex['text']}" if prefix else ex["text"]
    tok = tokenizer(text, truncation=True, max_length=256)
    tok["labels"] = int(ex["label"])  # assumes label is already 0/1/2
    return tok

train_ds = train_ds.map(encode, batched=False, remove_columns=train_ds.column_names)
eval_ds = eval_ds.map(encode, batched=False, remove_columns=eval_ds.column_names)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# -------- class weights --------
class_weights_tensor = None
if use_class_weights:
    counts = Counter(train_ds["labels"])
    print("Label counts in train:", counts)

    total = sum(counts.values())
    n_classes = num_labels
    weights = []
    for i in range(n_classes):
        c = counts.get(i, 1)
        w = total / (n_classes * c)
        weights.append(w)
    class_weights_tensor = torch.tensor(weights, dtype=torch.float)
    print("Class weights:", class_weights_tensor.tolist())

# -------- metrics --------
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = preds.argmax(axis=-1)
    acc = (preds == labels).mean()
    return {"accuracy": acc}

# -------- training args --------
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=float(config.get("learning_rate", 1e-4)),   # LoRA-friendly defaults
    per_device_train_batch_size=int(config.get("batch_size", 16)),
    num_train_epochs=float(config.get("epochs", 5)),
    # keep minimal since you're on an older transformers
)

trainer = WeightedTrainer(
    class_weights=class_weights_tensor,
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /content/drive/Shareddrives/cs685/mlm_bert_goemotions_biomed_finance and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Added tokens: 2 ['[FIN]', '[BIO]']


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


trainable params: 3,541,251 || all params: 113,027,334 || trainable%: 3.1331


Generating train split: 0 examples [00:00, ? examples/s]

...

 3


wandb: You chose "Don't visualize my results"


Step,Training Loss
500,1.035500
1000,0.975200
1500,0.955200
2000,0.908400
2500,0.895800
3000,0.884300
3500,0.831700
4000,0.823100
4500,0.834300
5000,0.766700


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:309: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

TrainOutput(global_step=15000, training_loss=0.7281048543294271, metrics={'train_runtime': 2499.22, 'train_samples_per_second': 96.03, 'train_steps_per_second': 6.002, 'total_flos': 2.30685713628048e+16, 'train_loss': 0.7281048543294271, 'epoch': 10.0})

In [ ]:
# --- Option A: merge LoRA into base and save a normal HF model ---
try:
    merged_model = trainer.model.merge_and_unload()
except AttributeError:
    # if using older peft that doesn't have merge_and_unload, just fall back
    merged_model = trainer.model

merged_model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("✅ Finished SFT (LoRA merged), saved to", output_dir)


✅ Finished SFT (LoRA merged), saved to /content/drive/MyDrive/models/sft7_lora


In [ ]:
!pip install -q transformers datasets safetensors scikit-learn


In [ ]:
from datasets import load_dataset

TEST_PATH = "/content/drive/Shareddrives/cs685/final_data_SFT/label_mixed_3_test.jsonl"  # or FIN/BIO test
test_ds = load_dataset("json", data_files=TEST_PATH, split="train")
len(test_ds), test_ds[0]


Generating train split: 0 examples [00:00, ? examples/s]

(3000,
 {'text': 'I have had the Implanon since August 2013 and had it removed yesterday at my GP. I have put on a ridiculous amount of weight use to be a size 6 since having it inserted I am now a 12. I couldnt cope with the irregular periods and they would last for two weeks. And then the Pre Menstrual Symptoms I would have them for about three weeks before my period which was horrible. I would cry all the time or be extremely moody',
  'label': 1,
  'domain': 'BIO'})

In [ ]:
def add_domain_prefix(example):
    domain = example.get("domain", None)
    if domain == "FIN":
        prefix = "[FIN]"
    elif domain == "BIO":
        prefix = "[BIO]"
    else:
        prefix = ""

    if prefix:
        return f"{prefix} {example['text']}"
    else:
        return example["text"]


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import json
from collections import defaultdict

# ---------- 1. Run evaluation ----------

texts = [add_domain_prefix(ex) for ex in test_ds]
labels = [int(ex["label"]) for ex in test_ds]
domains = [ex.get("domain", "UNK") for ex in test_ds]  # <- domain list

batch_size = 32
all_preds = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    inputs = tokenizer(
        batch_texts,
        return_tensors="pt",
        truncation=True,
        max_length=256,
        padding=True,
    ).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits
        preds = logits.argmax(dim=-1).cpu().tolist()

    all_preds.extend(preds)

labels = np.array(labels)
all_preds = np.array(all_preds)

acc = accuracy_score(labels, all_preds)
f1_macro = f1_score(labels, all_preds, average="macro")
f1_weighted = f1_score(labels, all_preds, average="weighted")

print(f"Accuracy:    {acc:.4f}")
print(f"Macro F1:    {f1_macro:.4f}")
print(f"Weighted F1: {f1_weighted:.4f}")

# ---------- 2. Per-domain metrics (FIN / BIO) ----------

for dom in ["FIN", "BIO"]:
    idx = [i for i, d in enumerate(domains) if d == dom]
    if not idx:
        continue
    dom_labels = labels[idx]
    dom_preds  = all_preds[idx]
    print(f"\nDomain: {dom}")
    print(f"  Accuracy: {accuracy_score(dom_labels, dom_preds):.4f}")
    print(f"  Macro F1: {f1_score(dom_labels, dom_preds, average='macro'):.4f}")

# ---------- 3. Collect misclassified indices ----------

mis_idx = np.where(labels != all_preds)[0]
print(f"\nTotal misclassified examples: {len(mis_idx)}")

# Group misclassified indices by domain
domain_to_mis_idx = defaultdict(list)
for idx in mis_idx:
    idx = int(idx)                     # ensure Python int
    dom = domains[idx]                 # use precomputed domains list
    domain_to_mis_idx[dom].append(idx)

for dom, idxs in domain_to_mis_idx.items():
    print(f"Domain {dom}: {len(idxs)} misclassified examples")

# ---------- 4. Stratified random sampling: 100 per domain ----------

np.random.seed(42)  # reproducible
error_samples = []

for dom, idxs in domain_to_mis_idx.items():
    n_available = len(idxs)
    n_sample = min(100, n_available)   # in case a domain has < 100 errors

    if n_sample == 0:
        continue

    chosen = np.random.choice(idxs, size=n_sample, replace=False)
    print(f"Sampling {n_sample} misclassified examples for domain {dom}")

    for idx in chosen:
        idx = int(idx)                 # again, Python int
        ex = test_ds[idx]

        item = {
            "dataset_index": idx,
            "text": ex["text"],              # original text, without prefix
            "true_label": int(ex["label"]),
            "pred_label": int(all_preds[idx]),
            "domain": dom,
            # fields for manual annotation
            "length_category": "",      # short / medium / long
            "has_negation": "",         # yes / no
            "sarcastic_or_ironic": "",  # yes / no
            "contains_numbers": "",     # yes / no
            "comment": "",              # free-form notes
        }
        error_samples.append(item)

print(f"\nTotal sampled for error analysis: {len(error_samples)}")

# ---------- 5. Save to JSON ----------

with open("error_analysis_sft7.json", "w", encoding="utf-8") as f:
    json.dump(error_samples, f, ensure_ascii=False, indent=2)

print("Saved stratified misclassified examples to error_analysis_sft7.json")


Accuracy:    0.6013
Macro F1:    0.5873
Weighted F1: 0.6039

Domain: FIN
  Accuracy: 0.7073
  Macro F1: 0.6370

Domain: BIO
  Accuracy: 0.4953
  Macro F1: 0.4956

Total misclassified examples: 1196
Domain BIO: 757 misclassified examples
Domain FIN: 439 misclassified examples
Sampling 100 misclassified examples for domain BIO
Sampling 100 misclassified examples for domain FIN

Total sampled for error analysis: 200
Saved stratified misclassified examples to error_analysis_sft7.json
